# Phase 2: Dimensionality Reduction & Embedding

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import umap

In [ ]:
# Load preprocessed data (from Phase 1)
df = pd.read_csv('preprocessed_campaign_data.csv')
df.head()

## Prepare Input Data (for autoencoder)

In [ ]:
# Select features for autoencoder (numeric + encoded categorical)
features = [
    'Income', 'Customer_Tenure', 'Total_Spend', 'Family_Size', 'Recency',
    'Education_encoded', 'Marital_Divorced', 'Marital_Married', 'Marital_Single',
    'Marital_Together', 'Marital_Widow'
]
X = df[features].values

# Train-test split (for validation)
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Standardize (reapply scaling to ensure consistency)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Build Autoencoder

In [ ]:
input_dim = X_train.shape[1]
encoding_dim = 10  # Latent space dimension (adjustable)

# Encoder
input_layer = Input(shape=(input_dim,))
encoder = Dense(64, activation='relu')(input_layer)
encoder = Dropout(0.2)(encoder)
encoder = Dense(32, activation='relu')(encoder)
latent_layer = Dense(encoding_dim, activation='relu')(encoder)

# Decoder
decoder = Dense(32, activation='relu')(latent_layer)
decoder = Dropout(0.2)(decoder)
decoder = Dense(64, activation='relu')(decoder)
output_layer = Dense(input_dim, activation='linear')(decoder)

# Compile
autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Train
history = autoencoder.fit(
    X_train, X_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, X_test),
    callbacks=[early_stopping],
    verbose=1
)


##  Extract Latent Embeddings

In [ ]:
encoder_model = Model(inputs=input_layer, outputs=latent_layer)
train_embeddings = encoder_model.predict(X_train)
test_embeddings = encoder_model.predict(X_test)

# Combine all embeddings
embeddings = np.vstack([train_embeddings, test_embeddings])

# Save embeddings for clustering
np.save('customer_embeddings.npy', embeddings)

## Visualize Embeddings

In [ ]:
# t-SNE (for 2D visualization)
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=embeddings_2d[:, 0], y=embeddings_2d[:, 1], alpha=0.6)
plt.title('t-SNE Visualization of Customer Embeddings')
plt.savefig('tsne_embeddings.png')  # For report
plt.show()

# UMAP (alternative to t-SNE)
umap_embeddings = umap.UMAP(random_state=42).fit_transform(embeddings)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=umap_embeddings[:, 0], y=umap_embeddings[:, 1], alpha=0.6)
plt.title('UMAP Visualization of Customer Embeddings')
plt.savefig('umap_embeddings.png')
plt.show()

## Documentation

In [ ]:
print("Autoencoder Training Summary:")
print(f"- Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"- Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print(f"- Embeddings saved to 'customer_embeddings.npy' (shape: {embeddings.shape})")